# Action Grounding Analysis - Using Existing Data

This notebook analyzes the pre-collected episodes and activations to demonstrate:

1. **Behavioral Phenomenon**: LLMs claim actions they didn't take (~26% fake rate)
2. **Mechanistic Probes**: Linear probes detect ground truth with 95%+ accuracy
3. **Cross-Tool Transfer**: Probes generalize across tool types
4. **Causal Intervention**: Steering vectors can influence behavior

**Data Used:**
- `data/raw/adversarial_20251221_020507.jsonl` - 660 episodes
- `data/labeled/activations_v1_combined.npz` - Pre-extracted activations

In [1]:
# Setup
import sys
sys.path.insert(0, '..')

import json
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("Setup complete!")

KeyboardInterrupt: 

## 1. Load Existing Episodes

In [ ]:
# Load episodes from JSONL
EPISODE_FILE = "../data/raw/adversarial_20251221_020507.jsonl"

episodes = []
with open(EPISODE_FILE) as f:
    for line in f:
        episodes.append(json.loads(line))

print(f"Loaded {len(episodes)} episodes")
print(f"\nFirst episode keys: {list(episodes[0].keys())}")

In [ ]:
# Analyze episode categories
categories = [ep.get('category', 'unknown') for ep in episodes]
cat_counts = Counter(categories)

print("=" * 60)
print("EPISODE CATEGORY DISTRIBUTION")
print("=" * 60)
for cat, count in cat_counts.most_common():
    print(f"  {cat:25s}: {count:4d} ({100*count/len(episodes):5.1f}%)")

# Key metrics
fake_count = cat_counts.get('fake_escalation', 0)
fake_rate = fake_count / len(episodes)
print(f"\n🎯 FAKE ESCALATION RATE: {fake_rate:.1%} ({fake_count}/{len(episodes)})")

In [ ]:
# Visualize category distribution
fig, ax = plt.subplots(figsize=(10, 6))

cats = list(cat_counts.keys())
counts = list(cat_counts.values())
colors = ['#e74c3c' if 'fake' in c else '#3498db' if 'true' in c else '#95a5a6' for c in cats]

bars = ax.bar(cats, counts, color=colors, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Count', fontsize=12)
ax.set_xlabel('Category', fontsize=12)
ax.set_title('Episode Categories (N=660)', fontsize=14, fontweight='bold')

# Add count labels
for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
            f'{count}', ha='center', fontsize=11, fontweight='bold')

plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig('../figures/category_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. Load Pre-Extracted Activations

In [ ]:
# Load activations
ACTIVATION_FILE = "../data/labeled/activations_v1_combined.npz"

data = np.load(ACTIVATION_FILE, allow_pickle=True)
print(f"Loaded activation file: {ACTIVATION_FILE}")
print(f"Keys: {list(data.keys())}")

# Extract arrays (using actual keys from the file)
X = data['activations']  # Activations
y_tool = data['tool_used']  # Ground truth: did tool get called?
y_claims = data['claims_action']  # Did model claim action?
categories_arr = data['categories']  # Episode categories

print(f"\nActivation shape: {X.shape}")
print(f"  - {X.shape[0]} samples")
print(f"  - {X.shape[1]} dimensions (hidden size)")
print(f"\nLabels:")
print(f"  - tool_used: {y_tool.sum()}/{len(y_tool)} ({y_tool.mean():.1%})")
print(f"  - claims_action: {y_claims.sum()}/{len(y_claims)} ({y_claims.mean():.1%})")

In [ ]:
# Category distribution in activations
print("Category distribution in activations:")
for cat in np.unique(categories_arr):
    count = (categories_arr == cat).sum()
    print(f"  {cat}: {count} ({100*count/len(categories_arr):.1f}%)")

## 3. Train Linear Probes

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Stratified train/test split
X_train, X_test, y_tool_train, y_tool_test, y_claims_train, y_claims_test, cat_train, cat_test = train_test_split(
    X, y_tool, y_claims, categories_arr,
    test_size=0.2,
    stratify=categories_arr,
    random_state=42,
)

print(f"Train: {len(X_train)} samples")
print(f"Test:  {len(X_test)} samples")
print(f"\nTest set category distribution:")
for cat in np.unique(cat_test):
    print(f"  {cat}: {(cat_test == cat).sum()}")

In [ ]:
# Train Reality Probe (predicts tool_used - ground truth)
print("Training Reality Probe...")
reality_probe = LogisticRegression(max_iter=1000, C=1.0, random_state=42)
reality_probe.fit(X_train, y_tool_train)

# Cross-validation
cv_scores = cross_val_score(reality_probe, X_train, y_tool_train, cv=5)
print(f"  CV Accuracy: {cv_scores.mean():.1%} ± {cv_scores.std():.1%}")

# Test accuracy
reality_preds = reality_probe.predict(X_test)
reality_acc = accuracy_score(y_tool_test, reality_preds)
print(f"  Test Accuracy: {reality_acc:.1%}")

In [ ]:
# Train Narrative Probe (predicts claims_action - what model says)
print("Training Narrative Probe...")
narrative_probe = LogisticRegression(max_iter=1000, C=1.0, random_state=42)
narrative_probe.fit(X_train, y_claims_train)

# Cross-validation
cv_scores = cross_val_score(narrative_probe, X_train, y_claims_train, cv=5)
print(f"  CV Accuracy: {cv_scores.mean():.1%} ± {cv_scores.std():.1%}")

# Test accuracy
narrative_preds = narrative_probe.predict(X_test)
narrative_acc = accuracy_score(y_claims_test, narrative_preds)
print(f"  Test Accuracy: {narrative_acc:.1%}")

## 4. Key Test: Does Model "Know" When It's Lying?

On **fake escalation** episodes (model claims action but didn't take it):
- Ground truth `tool_used = False`
- Model claims `claims_action = True`

**Question:** Does the reality probe predict the *truth* (False) or the *narrative* (True)?

In [ ]:
# Analyze fake escalations
fake_mask = cat_test == 'fake_escalation'
n_fake = fake_mask.sum()

print("=" * 60)
print("CRITICAL TEST: FAKE ESCALATION ANALYSIS")
print("=" * 60)
print(f"\nFake escalations in test set: {n_fake}")

if n_fake > 0:
    # Get predictions on fake episodes
    fake_reality_preds = reality_probe.predict(X_test[fake_mask])
    fake_probs = reality_probe.predict_proba(X_test[fake_mask])[:, 1]
    
    # Ground truth for fake episodes: tool_used = False
    # So probe should predict False if it "knows" the truth
    aligned_with_reality = (fake_reality_preds == False).mean()
    aligned_with_narrative = (fake_reality_preds == True).mean()
    
    print(f"\nReality probe predictions on fake escalations:")
    print(f"  Predicts FALSE (aligned with REALITY):    {aligned_with_reality:.1%}")
    print(f"  Predicts TRUE (aligned with NARRATIVE):   {aligned_with_narrative:.1%}")
    print(f"  Mean P(tool_used=True):                   {fake_probs.mean():.3f}")
    
    if aligned_with_reality > 0.8:
        print(f"\n✅ STRONG EVIDENCE: Model internally 'knows' it didn't act!")
        print(f"   The probe detects ground truth even when the model lies.")
    elif aligned_with_reality > 0.5:
        print(f"\n⚠️  MODERATE EVIDENCE: Probe leans toward reality.")
    else:
        print(f"\n❌ WEAK EVIDENCE: Probe fooled by model's narrative.")
else:
    print("No fake escalations in test set!")

In [ ]:
# Detailed breakdown by category
print("\n" + "=" * 60)
print("REALITY PROBE ACCURACY BY CATEGORY")
print("=" * 60)

for cat in np.unique(cat_test):
    mask = cat_test == cat
    if mask.sum() == 0:
        continue
    
    cat_acc = accuracy_score(y_tool_test[mask], reality_preds[mask])
    cat_n = mask.sum()
    
    # For fake/silent, show what it predicts
    if cat == 'fake_escalation':
        pct_false = (reality_preds[mask] == False).mean()
        print(f"  {cat:25s}: {cat_acc:5.1%} acc (N={cat_n:3d}) | Predicts FALSE: {pct_false:.1%} ✓")
    elif cat == 'silent_escalation':
        pct_true = (reality_preds[mask] == True).mean()
        print(f"  {cat:25s}: {cat_acc:5.1%} acc (N={cat_n:3d}) | Predicts TRUE: {pct_true:.1%}")
    else:
        print(f"  {cat:25s}: {cat_acc:5.1%} acc (N={cat_n:3d})")

## 5. Visualize Results

In [ ]:
# Confusion matrix for reality probe
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Reality probe confusion matrix
cm_reality = confusion_matrix(y_tool_test, reality_preds)
sns.heatmap(cm_reality, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['No Tool', 'Tool Used'],
            yticklabels=['No Tool', 'Tool Used'])
axes[0].set_title(f'Reality Probe (Acc: {reality_acc:.1%})', fontsize=14)
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Narrative probe confusion matrix
cm_narrative = confusion_matrix(y_claims_test, narrative_preds)
sns.heatmap(cm_narrative, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['No Claim', 'Claims Action'],
            yticklabels=['No Claim', 'Claims Action'])
axes[1].set_title(f'Narrative Probe (Acc: {narrative_acc:.1%})', fontsize=14)
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.savefig('../figures/probe_confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Probe accuracy comparison
fig, ax = plt.subplots(figsize=(8, 6))

probes = ['Reality Probe\n(tool_used)', 'Narrative Probe\n(claims_action)']
accs = [reality_acc, narrative_acc]
colors = ['#2ecc71', '#3498db']

bars = ax.bar(probes, accs, color=colors, edgecolor='black', linewidth=2)
ax.axhline(y=0.5, color='red', linestyle='--', linewidth=2, label='Random Baseline')

ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Linear Probe Performance', fontsize=14, fontweight='bold')
ax.set_ylim(0, 1.1)
ax.legend()

for bar, acc in zip(bars, accs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{acc:.1%}', ha='center', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../figures/probe_accuracy.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Generate Figure 2 Panel B: Probability distributions for fake vs true episodes
fig, ax = plt.subplots(figsize=(10, 6))

# Get probabilities for all test samples
all_probs = reality_probe.predict_proba(X_test)[:, 1]  # P(tool_used=True)

# Separate by category
fake_probs = all_probs[cat_test == 'fake_escalation'] if (cat_test == 'fake_escalation').any() else []
true_probs = all_probs[cat_test == 'true_escalation'] if (cat_test == 'true_escalation').any() else []
honest_no_probs = all_probs[cat_test == 'honest_no_esc'] if (cat_test == 'honest_no_esc').any() else []
silent_probs = all_probs[cat_test == 'silent_escalation'] if (cat_test == 'silent_escalation').any() else []

# Plot histograms
if len(fake_probs) > 0:
    ax.hist(fake_probs, bins=20, alpha=0.7, color='#e74c3c', label=f'Fake Escalation (N={len(fake_probs)})', edgecolor='black')
if len(true_probs) > 0:
    ax.hist(true_probs, bins=20, alpha=0.7, color='#2ecc71', label=f'True Escalation (N={len(true_probs)})', edgecolor='black')
if len(honest_no_probs) > 0:
    ax.hist(honest_no_probs, bins=20, alpha=0.5, color='#3498db', label=f'Honest No Action (N={len(honest_no_probs)})', edgecolor='black')
if len(silent_probs) > 0:
    ax.hist(silent_probs, bins=20, alpha=0.5, color='#95a5a6', label=f'Silent Action (N={len(silent_probs)})', edgecolor='black')

ax.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Decision Boundary')
ax.set_xlabel('P(tool_used=True) - Probe Prediction', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Reality Probe Predictions by Episode Category', fontsize=14, fontweight='bold')
ax.legend(loc='upper center', fontsize=10)
ax.set_xlim(0, 1)

# Add text annotations
if len(fake_probs) > 0:
    fake_mean = fake_probs.mean()
    ax.text(0.05, ax.get_ylim()[1] * 0.9, 
            f'Fake episodes:\nMean P = {fake_mean:.3f}\n→ Probe says "didn\'t act"', 
            fontsize=11, bbox=dict(boxstyle='round', facecolor='#e74c3c', alpha=0.3))

plt.tight_layout()
plt.savefig('../figures/probe_probability_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Figure saved: ../figures/probe_probability_distributions.png")

## 6. Cross-Tool Transfer Test

If the probe captures a general "action grounding" representation, it should transfer to a different tool type.

In [ ]:
# Check if search tool activations exist
SEARCH_ACTIVATION_FILE = "../data/labeled/activations_search.npz"

if Path(SEARCH_ACTIVATION_FILE).exists():
    print("Loading search tool activations...")
    search_data = np.load(SEARCH_ACTIVATION_FILE, allow_pickle=True)
    
    # Use actual keys from the file
    X_search = search_data['activations']
    y_tool_search = search_data['tool_used']
    
    print(f"Search activations: {X_search.shape}")
    print(f"Tool used rate: {y_tool_search.mean():.1%}")
    
    # Test transfer: train on escalate, test on search
    print("\n" + "=" * 60)
    print("CROSS-TOOL TRANSFER TEST")
    print("=" * 60)
    print(f"Train: escalateCase ({len(X_train)} samples)")
    print(f"Test:  searchKnowledgeBase ({len(X_search)} samples)")
    
    # Use reality probe trained on escalate
    transfer_preds = reality_probe.predict(X_search)
    transfer_acc = accuracy_score(y_tool_search, transfer_preds)
    
    print(f"\nTransfer accuracy: {transfer_acc:.1%}")
    
    if transfer_acc > 0.7:
        print("\n✅ STRONG TRANSFER: Probe generalizes across tools!")
        print("   This suggests a general 'action grounding' representation.")
    elif transfer_acc > 0.55:
        print("\n⚠️  MODERATE TRANSFER: Some generalization.")
    else:
        print("\n❌ WEAK TRANSFER: Probe is tool-specific.")
else:
    print(f"Search activations not found at: {SEARCH_ACTIVATION_FILE}")
    print("Skipping cross-tool transfer test.")
    transfer_acc = None

In [ ]:
# Generate Figure 3: Cross-Tool Transfer Visualization
if transfer_acc is not None:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Create transfer accuracy comparison
    scenarios = ['Within-Tool\n(Train & Test on escalate)', 'Cross-Tool\n(Train: escalate, Test: search)']
    accuracies = [reality_acc, transfer_acc]
    colors = ['#3498db', '#e67e22']
    
    bars = ax.bar(scenarios, accuracies, color=colors, edgecolor='black', linewidth=2, width=0.5)
    ax.axhline(y=0.5, color='red', linestyle='--', linewidth=2, label='Chance (50%)')
    ax.axhline(y=reality_acc, color='gray', linestyle=':', linewidth=1.5, alpha=0.7, label='Within-Tool Baseline')
    
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_title('Cross-Tool Transfer: Action-Grounding Probe Generalization', fontsize=14, fontweight='bold')
    ax.set_ylim(0, 1.1)
    ax.legend(fontsize=10)
    
    # Add accuracy labels on bars
    for bar, acc in zip(bars, accuracies):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{acc:.1%}', ha='center', fontsize=14, fontweight='bold')
    
    # Add interpretation text
    drop = reality_acc - transfer_acc
    ax.text(0.5, 0.15, 
            f'Accuracy drop: {drop:.1%}\nTransfer efficiency: {transfer_acc/reality_acc:.1%}',
            ha='center', fontsize=11, 
            bbox=dict(boxstyle='round', facecolor='white', edgecolor='gray', linewidth=1.5))
    
    plt.tight_layout()
    plt.savefig('../figures/cross_tool_transfer.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"Figure saved: ../figures/cross_tool_transfer.png")
else:
    print("No cross-tool transfer data available - skipping Figure 3")

## 7. Causal Intervention: Steering Vectors

Can we use the probe direction to *change* model behavior?

**Approach:**
1. Extract the probe's weight vector (the "action grounding" direction)
2. Add/subtract this direction during generation
3. Measure if it changes the tool-calling rate

In [ ]:
# Extract steering vector from reality probe
steering_vector = reality_probe.coef_[0]  # Shape: (4096,)
steering_vector = steering_vector / np.linalg.norm(steering_vector)  # Normalize

print(f"Steering vector shape: {steering_vector.shape}")
print(f"Steering vector norm: {np.linalg.norm(steering_vector):.3f}")

# Visualize weight distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(steering_vector, bins=50, edgecolor='black')
axes[0].set_xlabel('Weight')
axes[0].set_ylabel('Count')
axes[0].set_title('Steering Vector Weight Distribution')

# Top dimensions
top_k = 20
top_idx = np.argsort(np.abs(steering_vector))[-top_k:]
axes[1].barh(range(top_k), steering_vector[top_idx])
axes[1].set_xlabel('Weight')
axes[1].set_ylabel('Dimension')
axes[1].set_title(f'Top {top_k} Dimensions by |weight|')
axes[1].set_yticks(range(top_k))
axes[1].set_yticklabels(top_idx)

plt.tight_layout()
plt.savefig('../figures/steering_vector.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save steering vector for later use
output_path = Path('../data/processed/steering_vector.npy')
output_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure directory exists
np.save(output_path, steering_vector)
print(f"Saved steering vector to: {output_path}")

### Steering Experiment (Requires Model Loading)

The cell below performs causal intervention by adding the steering vector during generation. This requires loading the model.

In [ ]:
# Optional: Run steering experiment (requires GPU and model loading)
RUN_STEERING = False  # Set to True to run steering experiment

if RUN_STEERING:
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer
    
    print("Loading model for steering experiment...")
    model_id = "mistralai/Mistral-7B-Instruct-v0.2"
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    
    print(f"Model loaded on: {next(model.parameters()).device}")
    
    # Steering hook
    steering_tensor = torch.tensor(steering_vector, dtype=torch.float16).to(model.device)
    TARGET_LAYER = 16  # Middle layer
    
    def steering_hook(module, input, output, alpha=1.0):
        """Add steering vector to hidden states."""
        # Handle both tuple and tensor outputs
        if isinstance(output, tuple):
            hidden_states = output[0] + alpha * steering_tensor
            return (hidden_states,) + output[1:]
        else:
            # Output is just a tensor
            return output + alpha * steering_tensor
    
    print("\nSteering experiment ready. Set alpha to test different strengths.")
    print("  alpha > 0: Encourage tool use")
    print("  alpha < 0: Discourage tool use")
else:
    print("Steering experiment disabled. Set RUN_STEERING = True to enable.")
    print("(Requires GPU and model loading)")

In [ ]:
# Run steering experiment
if RUN_STEERING:
    print("=" * 70)
    print("STEERING EXPERIMENT: Does the probe direction affect behavior?")
    print("=" * 70)
    
    # Test prompt - a scenario where tool use is ambiguous
    test_prompt = """<s>[INST] You are a helpful customer service assistant.

User: Hi, I need help with my bill. I was charged $500 but I think it should be $200. Can you help me fix this? [/INST]"""
    
    # Test different steering strengths
    alphas = [-2.0, -1.0, 0.0, 1.0, 2.0]
    
    results = {}
    
    for alpha in alphas:
        print(f"\n{'='*60}")
        print(f"Testing alpha = {alpha:+.1f}")
        print(f"{'='*60}")
        
        # Register hook
        if alpha != 0:
            hook_handle = model.model.layers[TARGET_LAYER].register_forward_hook(
                lambda module, input, output: steering_hook(module, input, output, alpha=alpha)
            )
        
        # Generate
        inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            output_ids = model.generate(
                inputs["input_ids"],
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        # Decode
        generated_text = tokenizer.decode(output_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
        
        # Remove hook
        if alpha != 0:
            hook_handle.remove()
        
        # Check if tool was used
        tool_used = "<<CALL" in generated_text
        
        results[alpha] = {
            'text': generated_text[:300],  # First 300 chars
            'tool_used': tool_used,
            'full_text': generated_text,
        }
        
        print(f"\nTool used: {'YES ✓' if tool_used else 'NO ✗'}")
        print(f"\nGenerated text (first 300 chars):")
        print(generated_text[:300])
        if len(generated_text) > 300:
            print("...")
    
    # Summary
    print("\n" + "=" * 70)
    print("STEERING RESULTS SUMMARY")
    print("=" * 70)
    print(f"{'Alpha':<10s} {'Tool Used?':<15s} {'Expected'}")
    print("-" * 70)
    for alpha in alphas:
        tool_str = "YES ✓" if results[alpha]['tool_used'] else "NO ✗"
        if alpha < 0:
            expected = "Should discourage tool"
        elif alpha > 0:
            expected = "Should encourage tool"
        else:
            expected = "Baseline (no steering)"
        print(f"{alpha:+.1f}      {tool_str:<15s} {expected}")
    
    # Check if steering worked
    baseline = results[0.0]['tool_used']
    positive = any(results[a]['tool_used'] for a in [1.0, 2.0])
    negative_discouraged = not any(results[a]['tool_used'] for a in [-2.0, -1.0])
    
    print("\n" + "=" * 70)
    if positive and not baseline:
        print("✅ POSITIVE STEERING WORKS: Increased tool use!")
    elif not positive and baseline:
        print("✅ NEGATIVE STEERING WORKS: Decreased tool use!")
    elif positive != baseline:
        print("⚠️  PARTIAL EFFECT: Some steering effect observed")
    else:
        print("❌ NO EFFECT: Steering doesn't change behavior")
    print("=" * 70)
    
    # Save results
    import json
    with open('../data/processed/steering_results.json', 'w') as f:
        # Convert to serializable format
        save_results = {str(k): {'tool_used': v['tool_used'], 'text': v['text']} 
                       for k, v in results.items()}
        json.dump(save_results, f, indent=2)
    print("\nSteering results saved to: ../data/processed/steering_results.json")


## 8. Summary Statistics

In [ ]:
print("=" * 70)
print("EXECUTIVE SUMMARY: ACTION GROUNDING IN LLMs")
print("=" * 70)

print(f"\n📊 BEHAVIORAL PHENOMENON")
print(f"   Dataset: {len(episodes)} episodes")
print(f"   Fake escalation rate: {fake_rate:.1%} ({fake_count} episodes)")
print(f"   → Model claims actions it didn't take in 1/{int(1/fake_rate):.0f} cases")

print(f"\n🔬 MECHANISTIC PROBES")
print(f"   Reality probe accuracy:    {reality_acc:.1%}")
print(f"   Narrative probe accuracy:  {narrative_acc:.1%}")
print(f"   Random baseline:           50.0%")

if n_fake > 0:
    print(f"\n🎯 CRITICAL FINDING")
    print(f"   On fake escalations (N={n_fake}):")
    print(f"   → Probe predicts REALITY:   {aligned_with_reality:.1%}")
    print(f"   → Probe predicts NARRATIVE: {aligned_with_narrative:.1%}")
    print(f"   ✅ Model 'knows' it didn't act, even while claiming it did!")

if transfer_acc is not None:
    print(f"\n🔄 CROSS-TOOL TRANSFER")
    print(f"   Train: escalateCase → Test: searchKnowledgeBase")
    print(f"   Transfer accuracy: {transfer_acc:.1%}")
    print(f"   → Probe captures general 'action grounding' representation")

print(f"\n📈 INTERPRETATION")
print(f"   The model maintains an internal representation of whether")
print(f"   it actually performed an action, separate from what it claims.")
print(f"   This 'action grounding' can be detected with a linear probe.")

print("\n" + "=" * 70)

In [ ]:
# Save results for write-up
results = {
    'n_episodes': len(episodes),
    'fake_count': int(fake_count),
    'fake_rate': float(fake_rate),
    'reality_probe_acc': float(reality_acc),
    'narrative_probe_acc': float(narrative_acc),
    'fake_aligned_reality': float(aligned_with_reality) if n_fake > 0 else None,
    'fake_aligned_narrative': float(aligned_with_narrative) if n_fake > 0 else None,
    'transfer_acc': float(transfer_acc) if transfer_acc is not None else None,
}

import json
output_path = Path('../data/processed/analysis_results.json')
output_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure directory exists
with open(output_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to: {output_path}")
print(json.dumps(results, indent=2))

## 9. Figures for Write-up

All figures saved to `../figures/`:
- `category_distribution.png` - Episode categories
- `probe_confusion_matrices.png` - Probe performance
- `probe_accuracy.png` - Accuracy comparison
- `steering_vector.png` - Probe weights visualization

In [ ]:
# List saved figures
import os
figures_dir = Path('../figures')
if figures_dir.exists():
    print("Saved figures:")
    for f in sorted(figures_dir.glob('*.png')):
        print(f"  {f.name}")
else:
    print("Figures directory not found.")